In [1]:
import numpy as np # type: ignore
import pandas as pd # type: ignore

import pickle as pkl
import plotly.express as px # type: ignore
import plotly.graph_objects as go # type: ignore

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from StreamlitApp.functions.carga_dataframes import *
from ML.escalado_datos import *
from StreamlitApp.passwords import pw

from sklearn.model_selection import train_test_split # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import GRU, Dense, Input, Dropout # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore

2025-03-05 11:40:39.051223: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 11:40:39.054682: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 11:40:39.068469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741171239.088446   23561 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741171239.094477   23561 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 11:40:39.115333: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# Cargar el scaler
with open("../data/data_scaled/scalers/scaler_consumo_anio_DF_DEMANDA.pkl", "br") as file:
    scaler = pkl.load(file)

In [3]:
# Función para crear secuencias
def crea_secuencias(dataframe, target_col, len_secuencia) -> tuple:
    X, y = [], []
    for i in range(len(dataframe) - len_secuencia):
        X.append(dataframe.iloc[i:i+len_secuencia].drop(columns=[target_col]).values) 
        y.append(dataframe.iloc[i+len_secuencia][target_col]) 
    return np.array(X), np.array(y)

In [4]:
# Función para redimensionar las secuencias
def redimensiona(xtrain, xtest, ventana, n_features) -> str:
    xtrain = xtrain.reshape((xtrain.shape[0], ventana, n_features))  
    xtest = xtest.reshape((xtest.shape[0], ventana, n_features)) 
    return f"X_train shape: {xtrain.shape}, X_test shape: {xtest.shape}"

In [5]:
'''# Construcción del modelo GRU básico
def crea_gru(input_shape, len_secuencia, xtrain, xtest, ytrain, ytest) -> tuple:
    model = Sequential([
        Input(shape=(len_secuencia, input_shape)),
        GRU(64, activation='tanh'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=50, verbose=0, callbacks=[early_stopping])
    return model, history'''

"# Construcción del modelo GRU básico\ndef crea_gru(input_shape, len_secuencia, xtrain, xtest, ytrain, ytest) -> tuple:\n    model = Sequential([\n        Input(shape=(len_secuencia, input_shape)),\n        GRU(64, activation='tanh'),\n        Dense(1, activation='linear')\n    ])\n    model.compile(optimizer='adam', loss='mse', metrics=['mae'])\n    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)\n    history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=50, verbose=0, callbacks=[early_stopping])\n    return model, history"

In [6]:
'''# Construcción del modelo GRU con más capas. Este modelo me parece más realista
def crea_gru(input_shape, len_secuencia, xtrain, xtest, ytrain, ytest) -> tuple:
    model = Sequential([
        Input(shape=(len_secuencia, input_shape)),
        GRU(64, activation='tanh', return_sequences=True),  
        Dropout(0.3), 
        GRU(64, activation='tanh', return_sequences=False),  
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='linear') 
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=50, verbose=0, callbacks=[early_stopping])
    return model, history'''

"# Construcción del modelo GRU con más capas. Este modelo me parece más realista\ndef crea_gru(input_shape, len_secuencia, xtrain, xtest, ytrain, ytest) -> tuple:\n    model = Sequential([\n        Input(shape=(len_secuencia, input_shape)),\n        GRU(64, activation='tanh', return_sequences=True),  \n        Dropout(0.3), \n        GRU(64, activation='tanh', return_sequences=False),  \n        Dense(64, activation='relu'),\n        Dropout(0.3),\n        Dense(1, activation='linear') \n    ])\n    model.compile(optimizer='adam', loss='mse', metrics=['mae'])\n    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)\n    history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=50, verbose=0, callbacks=[early_stopping])\n    return model, history"

In [7]:
# Construcción del modelo GRU con más capas. Este modelo me parece más realista
def crea_gru(input_shape, len_secuencia, xtrain, xtest, ytrain, ytest) -> tuple:
    model = Sequential([
        Input(shape=(len_secuencia, input_shape)),
        GRU(64, activation='tanh', return_sequences=True),  
        Dropout(0.3), 
        GRU(64, activation='tanh', return_sequences=False),  
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='linear') 
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=50, verbose=0, callbacks=[early_stopping])
    if len_secuencia == 7:
        with open("MODELS/GRU/gru_model7.pkl", "bw") as file:
            pickle.dump(model,file)
            return model, history
    elif len_secuencia == 15:
        with open("MODELS/GRU/gru_model15.pkl", "bw") as file:
            pickle.dump(model,file)
            return model, history
    if len_secuencia == 30:
        with open("MODELS/GRU/gru_model30.pkl", "bw") as file:
            pickle.dump(model,file)
            return model, history

In [8]:
# Función para graficar loss y mae
def grafica_loss_mae(historial) -> None:
    fig = px.line(data_frame=historial.history, 
                  y=['loss', 'val_loss'], 
                  title='Función de pérdida (loss) basada en el MSE',
                  labels={'index': 'Época', 'value': 'Pérdida'})
    
    fig.update_layout(title_x=0.5, 
                      template="plotly_white",
                      legend_title_text="Variables")
    
    fig.for_each_trace(lambda x: x.update(name="Pérdida Entrenamiento" if x.name == "loss" else "Pérdida Validación"))

    fig.show()
    return None

In [9]:
# Función para predecir 1-step
def predice_1step(model, data, y, scaler, len_secuencia, num_dias) -> tuple:
    validation_target = y[-num_dias:]
    validation_predictions = []
    
    i = -num_dias
    while len(validation_predictions) < len(validation_target):
        p = model.predict(data[i].reshape(1, len_secuencia, data.shape[2]))[0, 0]
        validation_predictions.append(p)
        i += 1

    # Crear arrays para revertir el escalado
    dummy_features = np.zeros((len(validation_predictions), 1))
    predictions_with_dummy = np.hstack([np.array(validation_predictions).reshape(-1, 1), dummy_features])
    validation_target_dummy = np.hstack([validation_target.reshape(-1, 1), dummy_features])

    # Desescalar
    predictions_desescalado = scaler.inverse_transform(predictions_with_dummy)[:, 0]
    validation_desescalado = scaler.inverse_transform(validation_target_dummy)[:, 0]

    return predictions_desescalado, validation_desescalado

In [10]:
# Función para predecir multi-step
def predice_multistep(model, data, scaler, len_secuencia, num_dias_multi) -> np.array:
    predictions = []
    input_seq = data[-1].reshape(1, len_secuencia, data.shape[2])
    
    for _ in range(num_dias_multi):
        pred = model.predict(input_seq)[0, 0]
        predictions.append(pred)
        input_seq = np.roll(input_seq, shift=-1, axis=1)  # Desplazar la secuencia
        input_seq[0, -1, -1] = pred  # Actualizar con la predicción
    
    # Desescalar
    dummy_features = np.zeros((len(predictions), 1))
    predictions_with_dummy = np.hstack([np.array(predictions).reshape(-1, 1), dummy_features])
    predictions_desescalado = scaler.inverse_transform(predictions_with_dummy)[:, 0]

    return predictions_desescalado

In [11]:
# Función para predecir x días en el futuro
def predice_futuro(model, data, scaler, len_secuencia, num_dias_futuro) -> np.array:
    predictions = []
    input_seq = data[-1].reshape(1, len_secuencia, data.shape[2])

    for _ in range(num_dias_futuro):
        # Generar predicción
        pred = model.predict(input_seq, verbose=0)[0, 0]
        predictions.append(pred)
        
        # Actualizar la secuencia de entrada con la predicción
        input_seq = np.roll(input_seq, shift=-1, axis=1)
        input_seq[0, -1, -1] = pred

    # Desescalar las predicciones
    dummy_features = np.zeros((len(predictions), 1))
    predictions_with_dummy = np.hstack([np.array(predictions).reshape(-1, 1), dummy_features])
    predictions_desescalado = scaler.inverse_transform(predictions_with_dummy)[:, 0]

    return predictions_desescalado

In [12]:
# Graficar las predicciones 1-step y multi-step
def grafica_predicciones(real, pred_1step, pred_multistep) -> None:
    fig = go.Figure()

    # Valores reales
    fig.add_trace(go.Scatter(
        y=real,
        mode='lines+markers',
        name='Valores Reales',
        line=dict(color='blue', width=2),
        marker=dict(size=6)
    ))

    # Predicciones 1-step
    fig.add_trace(go.Scatter(
        y=pred_1step,
        mode='lines+markers',
        name=f'Predicciones 1-step',
        line=dict(color='red', width=2, dash='dot'),
        marker=dict(size=6)
    ))

    # Predicciones multi-step
    fig.add_trace(go.Scatter(
        y=pred_multistep,
        mode='lines+markers',
        name=f'Predicciones Multi-step',
        line=dict(color='green', width=2, dash='dot'),
        marker=dict(size=6)
    ))

    # Configuración de la gráfica
    fig.update_layout(
        title="Predicciones vs Valores Reales",
        title_x=0.5,
        xaxis_title="Días",
        yaxis_title="Demanda (GWh)",
        template="plotly_white",
        hovermode="x",
        xaxis=dict(
            tickvals=list(range(len(pred_1step) + 1)),  
            ticktext=[str(i) for i in range(1, len(pred_1step) + 1)]  
        ))

    fig.show()
    return None

In [13]:
# Graficar las predicciones futuras
def grafica_predicciones_futuras(pred_futuro) -> None:
    fig = go.Figure()

    # Predicciones futuras
    fig.add_trace(go.Scatter(
        y=pred_futuro,
        mode='lines+markers',
        name=f'Predicciones Futuras',
        line=dict(color='orange', width=2, dash='dot'),
        marker=dict(size=6)
    ))

    # Configuración de la gráfica
    fig.update_layout(
        title="Predicciones para Días Futuros",
        title_x=0.5,
        xaxis_title="Días Futuros",
        yaxis_title="Demanda (GWh)",
        template="plotly_white",
        hovermode="x",
        xaxis=dict(
            tickvals=list(range(len(pred_futuro))),
            ticktext=[str(i + 1) for i in range(len(pred_futuro))]
        ))

    fig.show()
    return None

In [14]:
# Cargar los datos y procesarlos
df_demanda = carga_dataframes(pw["host"], pw["user"], pw["password"], pw["database"])[1]
df = procesar_datos(df_demanda)
df = df.drop(columns="fecha")
TARGET = df["valor_(GWh)"]

# Configuración inicial
for ventana in [7, 15, 30]:
    n_features = len([col for col in df.columns if col != TARGET.name])
    X, y = crea_secuencias(df, TARGET.name, ventana)

    print("Forma de X:", X.shape)
    print("Forma de y:", y.shape)

    # División en entrenamiento y test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)  # No barajar los datos
    redimensiona(X_train, X_test, ventana, n_features)

    # Construir y entrenar el modelo
    model, history = crea_gru(X.shape[2], ventana, X_train, X_test, y_train, y_test)
    grafica_loss_mae(history)

    # Predicciones 1-step
    pred_1step, real_1step = predice_1step(model, X_test, y_test, scaler, ventana, num_dias=ventana)

    # Predicciones multi-step
    pred_multistep = predice_multistep(model, X_test, scaler, ventana, num_dias_multi=ventana)

Columnas antes de procesar: Index(['fecha', 'año', 'mes', 'dia', 'dia_semana', 'valor_(GWh)',
       'es_festivo'],
      dtype='object')
Procesamiento completado. Datos preparados y guardados.
Forma de X: (5168, 7, 8)
Forma de y: (5168,)


2025-03-05 11:41:02.845619: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Forma de X: (5160, 15, 8)
Forma de y: (5160,)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━

In [15]:
# Graficar predicciones
grafica_predicciones(real_1step, pred_1step, pred_multistep)

In [16]:
# Predicción y graficado de días futuros (los mismos que ventana temporal)
pred_futuro = predice_futuro(model, X_test, scaler, ventana, num_dias_futuro=ventana)

grafica_predicciones_futuras(pred_futuro)